# Prueba del modelo

In [1]:
from torchvision.models import efficientnet_b0
import torch.nn as nn

class FramewiseEfficientNet(nn.Module):
    def __init__(self, sequence_length=16, dropout=0.4):
        super(FramewiseEfficientNet, self).__init__()
        self.backbone = efficientnet_b0(weights='IMAGENET1K_V1')
        self.backbone.classifier = nn.Identity()
        self.classifier = nn.Sequential(
            nn.Linear(1280, 512),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(512, 1)
        )

    def forward(self, x):  # x: [B, T, C, H, W]
        B, T, C, H, W = x.shape
        x = x.view(B * T, C, H, W)
        feats = self.backbone(x)
        feats = feats.view(B, T, -1).mean(1)
        return self.classifier(feats).view(-1)


In [37]:
from torchvision import transforms
import face_recognition
import cv2
import numpy as np
import torch

# Configurar device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Cargar modelo
model = FramewiseEfficientNet()
model.load_state_dict(torch.load("modelo_finetuneado3.pt"))
model.to(device)
model.eval()

# Configuración de transform
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225]),
])

def preprocess_video_for_inference(video_path, num_frames=16):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Índices espaciados
    indices = np.linspace(0, total_frames - 1, num_frames, dtype=int)
    frames = []

    for i in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break
        if i in indices:
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            faces = face_recognition.face_locations(rgb)
            if faces:
                top, right, bottom, left = faces[0]
                face = rgb[top:bottom, left:right]
                face = cv2.resize(face, (224, 224))
                tensor = transform(face)
                frames.append(tensor)

    cap.release()

    if len(frames) < num_frames:
        print(f"⚠️ Solo se detectaron {len(frames)} rostros. Rellenando...")
        while len(frames) < num_frames:
            frames.append(frames[-1].clone())

    video_tensor = torch.stack(frames).unsqueeze(0)  # [1, T, C, H, W]
    return video_tensor

# Evaluar un video
video_path = "C:/Users/Hermanos/Desktop/Proyecto Deepfake/prueba28.mp4"
video_tensor = preprocess_video_for_inference(video_path).to(device)

with torch.no_grad():
    output = model(video_tensor)
    prob = torch.sigmoid(output).item()
    print(f"📹 Resultado para video: {prob:.4f} ({'Fake' if prob > 0.5 else 'Real'})")


📹 Resultado para video: 0.9211 (Fake)


In [ ]:
import torch

# Configurar device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Cargar modelo
model = FramewiseEfficientNet()
model.load_state_dict(torch.load("modelo_finetuneado3.pt"))
model.to(device)
model.eval()

# Ruta a un tensor del test set
pt_path = "C:/Users/Hermanos/Desktop/Proyecto Deepfake/preprocesados/Face2Face_029_048.pt"
tensor = torch.load(pt_path).unsqueeze(0).to(device)  # [1, T, C, H, W]

# Evaluación
with torch.no_grad():
    output = model(tensor)
    prob = torch.sigmoid(output).item()
    print(f"🔍 Resultado para .pt: {prob:.4f} ({'Fake' if prob > 0.5 else 'Real'})")


🔍 Resultado para .pt: 1.0000 (Fake)
